<a href="https://colab.research.google.com/github/kluo9/HM-personalized-fashion-recommendation/blob/main/hm_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of ranking stage is to give rankings of the items recalled based on features from item and customer. 

The training is achieved with input as features from item and customers, and out = 1/0 based on whether this item is purchased ot not.

This becomes a classification problem, and it is very imbalanced since the items with output label 1 is very small compared to 0. So I generate the negative samples (out label = 0) so that the negative sample size is about 30 * the positive (out label = 1) sample size.


In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
with open("/content/gdrive/MyDrive/hm_recommendation/user_GT_vali", "rb") as fp: 
     user_vali_GT = pickle.load(fp)

In [6]:
with open("/content/gdrive/MyDrive/hm_recommendation/user_recall_vali", "rb") as fp: 
     user_vali_recall = pickle.load(fp)

# Featue engineering

In [10]:
# read item embedding
with open("/content/gdrive/MyDrive/hm_recommendation/item_embedding", "rb") as fp: 
     item_embedding = pickle.load(fp)